In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
import keras
import os

2024-05-14 02:30:51.380204: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-05-14 02:30:51.758542: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-05-14 02:30:51.758610: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-05-14 02:30:51.820485: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-05-14 02:30:52.018557: I tensorflow/core/platform/cpu_feature_guar

In [2]:
model_version = '1.0'
# latent factor의 수
num_features = 30
# iteration 수
iterations = 140
# learning_rate 
learning_rate = 1e-1
# lambda
lambda_ = 1
# count_weight
count_weight = 20

In [3]:
# 저장된 모델 버전 및 체크포인트 경로 설정
checkpoint_path = f'../model/CF/{model_version}/'

# CSV 파일 불러오기
UI_temp = pd.read_csv(checkpoint_path + 'UI_temp.csv').drop('userId', axis=1)
UI_count_div = pd.read_csv(checkpoint_path + 'UI_count_div.csv').drop('userId', axis=1)
test_for_loss = pd.read_csv(checkpoint_path + 'test_for_loss.csv')
test_data_df = pd.read_csv(checkpoint_path + 'test_data_df.csv')
user_category_not_valid_df = pd.read_csv(checkpoint_path + 'user_category_not_valid.csv')
category_df = pd.read_csv(checkpoint_path + 'category.csv')

In [4]:
bins = category_df.values[0, 1:]
labels = category_df.columns[1:]

In [5]:
# labels의 dtype을 float64로 변경
labels = labels.astype('float64')

In [6]:
# CF를 위한 초기값 설정
Y = np.array(UI_temp) 
Y = Y.T
count = np.array(UI_count_div)
count = count.T
print(Y.shape)
R = Y != 0 
n_u = Y.shape[1]
n_o = Y.shape[0]

(696, 14)


In [7]:
# 기록이 존재하는 값의 평균을 구함
o_sum = Y.sum(axis=1)
o_count = R.sum(axis=1)
o_mean = o_sum / o_count
o_mean = o_mean.reshape(-1, 1)

In [8]:
Y_stand = Y - (o_mean * R)

In [9]:
# user, outfit의 수
n_o, n_u = Y.shape

# (U,O)를 초기화하고 tf.Variable로 등록하여 추적
tf.random.set_seed(1234) # for consistent results
U = tf.Variable(tf.random.normal((n_u,  num_features),dtype=tf.float64),  name='U')
O = tf.Variable(tf.random.normal((n_o, num_features),dtype=tf.float64),  name='O')
b = tf.Variable(tf.random.normal((1,          n_u),   dtype=tf.float64),  name='b')

# optimizer 초기화
optimizer = keras.optimizers.Adam(learning_rate = learning_rate)

2024-05-14 02:30:54.781093: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-05-14 02:30:55.209227: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-05-14 02:30:55.209273: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-05-14 02:30:55.215076: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-05-14 02:30:55.215132: I external/local_xla/xla/stream_executor

In [10]:
def load_variables_optimizer(variables, optimizer, checkpoint_path):
    checkpoint = tf.train.Checkpoint(variables=variables, optimizer=optimizer)
    checkpoint.restore(tf.train.latest_checkpoint(checkpoint_path))

In [11]:
# 훈련된 변수 불러오기
load_variables_optimizer({"O": O, "U": U, "b": b}, optimizer,  checkpoint_path)

In [12]:
O

<tf.Variable 'O:0' shape=(696, 30) dtype=float64, numpy=
array([[ 0.0003777 , -0.00590412,  0.0019217 , ..., -0.00245451,
        -0.00017366,  0.00505472],
       [-0.00038296, -0.00565587,  0.00185378, ..., -0.00129054,
        -0.00178141,  0.00539561],
       [-0.00095602, -0.00346697,  0.00042633, ...,  0.00296692,
        -0.00146871,  0.00343953],
       ...,
       [-0.00346979,  0.00183975,  0.0008086 , ...,  0.00017552,
         0.00282561, -0.00231453],
       [-0.00124138, -0.00508827,  0.00059356, ...,  0.00228199,
        -0.00126017,  0.00360561],
       [ 0.0010426 , -0.0005276 ,  0.00076045, ..., -0.00083188,
         0.00085243, -0.00150309]])>

In [13]:
def cofi_cost_func_v(O, U, b, Y, R, lambda_):
    j = (tf.linalg.matmul(O, tf.transpose(U)) + b - Y )*R
    J = 0.5 * tf.reduce_sum(j**2) + (lambda_/2) * (tf.reduce_sum(O**2) + tf.reduce_sum(U**2))
    return J

In [14]:
def test(O, U, b, o_mean, count, count_weight, df, UI_temp, labels, user_category_not_valid) :
    # 예측을 수행하기 위해 모든 user-item에 대한 예측값을 계산
    p = np.matmul(O.numpy(), np.transpose(U.numpy())) + b.numpy()
    # user_category_not_valid에 해당하지 않는 경우에 대해 precision, recall, f1_score 계산
    # 평균을 위한 초기화
    precision_m, recall_m, f1_score_m, count_m = 0, 0, 0, 0
    for i in range(UI_temp.shape[0]):
        for category in labels:
            
            # 실제 온도
            # 평균을 적용하고 temp를 빼서 값이 작을수록 실제 온도에 가깝도록 함. 이 때 각 user-item의 사용 횟수를 가중하여 많이 사용한 item이 추천되도록 함
            pm = np.power(p + o_mean - category, 2)  -count * count_weight
            my_predictions = pm[:,i]

            # sort predictions
            ix = tf.argsort(my_predictions, direction='ASCENDING')

            df_predict = UI_temp[UI_temp.columns[ix[0:3]]].copy()
            df_predict = df_predict.round(0)
            # df_predict의 columns와 test_data_df의 '옷 조합' column을 비교하여 일치하는 경우의 개수를 계산
            predict = df_predict.columns.astype(str)
            
            if i+1 in user_category_not_valid and category in user_category_not_valid[i+1]:
                continue
            
            label = df[(df['userId'] == i+1) & (df['평균기온(°C)'] == category)]['옷 조합'].astype(str)
            # label이 UI_temp의 column에 포함되지 않는다면 제외
            label = label[label.isin(UI_temp.columns)]
            # label에 어떠한 옷 조합도 포함되지 않을 시 지표를 측정하지 않음
            if label.shape[0] == 0:
                continue
            
            count_m += 1
            precision = len(set(predict) & set(label)) / len(set(predict))
            recall = len(set(predict) & set(label)) / len(set(label))
            if precision + recall == 0: 
                f1_score = 0
            else:
                f1_score = 2 * (precision * recall) / (precision + recall)
            precision_m += precision
            recall_m += recall
            f1_score_m += f1_score
    precision_m /= count_m
    recall_m /= count_m
    f1_score_m /= count_m
    return precision_m, recall_m, f1_score_m

In [15]:
precision, recall, f1_score = test(O, U, b, o_mean, count, count_weight, test_data_df, UI_temp, labels, user_category_not_valid_df)

2024-05-14 02:31:00.096888: I external/local_tsl/tsl/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory


In [16]:
print(f'평균 precision: {precision}, 평균 recall: {recall}, 평균 f1_score: {f1_score}')

평균 precision: 0.42028985507246375, 평균 recall: 0.16773262968915145, 평균 f1_score: 0.2365935996370779


In [ ]:
data = {'precision': [precision], 'recall': [recall], 'f1_score': [f1_score]}

In [ ]:
json_data = json.dumps(data)
print(json_data)